In [135]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [136]:
df_train = pd.read_csv('./train.csv', sep=";")

In [137]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 21 columns):
age               2999 non-null int64
job               2999 non-null object
marital           2999 non-null object
education         2999 non-null object
default           2999 non-null object
housing           2999 non-null object
loan              2999 non-null object
contact           2999 non-null object
month             2999 non-null object
day_of_week       2999 non-null object
duration          2999 non-null int64
campaign          2999 non-null int64
pdays             2999 non-null int64
previous          2999 non-null int64
poutcome          2999 non-null object
emp.var.rate      2999 non-null float64
cons.price.idx    2999 non-null float64
cons.conf.idx     2999 non-null float64
euribor3m         2999 non-null float64
nr.employed       2999 non-null float64
y                 2999 non-null object
dtypes: float64(5), int64(5), object(11)
memory usage: 492.1+ KB


In [138]:
df_train = df_train.drop(['month','day_of_week','poutcome'],axis=1)

In [139]:
# no : 0, yes: 1,unknown:-1
df_train['default'] = df_train['default'].map({'no':0,'yes':1,'unknown':-1})
df_train['housing'] = df_train['housing'].map({'no':0,'yes':1,'unknown':-1})
df_train['loan'] = df_train['loan'].map({'no':0,'yes':1,'unknown':-1})
df_train['y'] = df_train['y'].map({'no':0,'yes':1,'unknown':-1})
df_train['contact'] = df_train['contact'].map({'cellular':0,'telephone':1})
df_train['marital'] = df_train['marital'].map({'divorced':0,'married':1, 'single' : 2, 'unknown': -1})
df_train['education'] = df_train['education'].map({'basic.4y':0,'basic.6y':1,'basic.9y':2,'high.school':3,'illiterate':4,'professional.course':5,'university.degree':6,'unknown':-1})
df_train['job'] = df_train['job'].map({'admin.':0,'blue-collar':1,'entrepreneur':2,'housemaid':3,'management':4,'retired':5,'self-employed':6,'services':7,'student':8,'technician':9,'unemployed':10,'unknown':-1})

In [140]:
df_train

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,1,1,2,0,1,0,0,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,7,2,3,0,0,0,1,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,0
2,25,7,1,3,0,1,0,1,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,0
3,38,7,1,2,0,-1,-1,1,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,0
4,47,0,1,6,0,1,0,0,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,0
5,32,7,2,6,0,0,0,0,128,3,999,2,-1.1,94.199,-37.5,0.884,4963.6,0
6,32,0,2,6,0,1,0,0,290,4,999,0,-1.1,94.199,-37.5,0.879,4963.6,0
7,41,2,1,6,-1,1,0,0,44,2,999,0,-0.1,93.200,-42.0,4.191,5195.8,0
8,31,7,0,5,0,0,0,0,68,1,999,1,-0.1,93.200,-42.0,4.153,5195.8,0
9,35,1,1,2,-1,0,0,1,170,1,999,0,1.1,93.994,-36.4,4.855,5191.0,0


In [141]:
df_test = pd.read_csv('./test.csv', sep=";",  header=None)

In [142]:
df_test.columns = ['age','job','marital','education','default','housing','loan','contact','month','day_of_week','duration','campaign','pdays','previous','poutcome','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']

In [143]:
df_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,49,management,divorced,university.degree,no,no,no,cellular,jul,thu,144,5,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1
1,31,admin.,unknown,high.school,no,no,no,telephone,may,mon,182,8,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0
2,31,admin.,single,basic.9y,no,no,no,telephone,may,wed,80,1,999,1,failure,-1.8,92.893,-46.2,1.281,5099.1
3,60,admin.,married,high.school,no,yes,yes,cellular,apr,thu,482,2,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1
4,33,blue-collar,married,high.school,no,no,no,cellular,apr,mon,87,2,999,2,failure,-1.8,93.075,-47.1,1.405,5099.1
5,41,self-employed,married,university.degree,no,yes,no,telephone,may,mon,796,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
6,45,services,married,professional.course,no,yes,no,telephone,may,mon,221,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
7,29,services,divorced,high.school,no,no,yes,telephone,may,mon,251,11,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
8,39,unemployed,married,university.degree,no,no,no,telephone,jun,tue,132,2,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
9,35,admin.,married,high.school,unknown,yes,yes,telephone,may,tue,236,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


In [144]:
df_test = df_test.drop(['month','day_of_week','poutcome'],axis=1)

In [145]:
df_test.columns = range(df_test.shape[1])
df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,49,management,divorced,university.degree,no,no,no,cellular,144,5,999,0,1.4,93.918,-42.7,4.963,5228.1
1,31,admin.,unknown,high.school,no,no,no,telephone,182,8,999,0,1.1,93.994,-36.4,4.858,5191.0
2,31,admin.,single,basic.9y,no,no,no,telephone,80,1,999,1,-1.8,92.893,-46.2,1.281,5099.1
3,60,admin.,married,high.school,no,yes,yes,cellular,482,2,999,0,-1.8,93.075,-47.1,1.365,5099.1
4,33,blue-collar,married,high.school,no,no,no,cellular,87,2,999,2,-1.8,93.075,-47.1,1.405,5099.1
5,41,self-employed,married,university.degree,no,yes,no,telephone,796,1,999,0,1.1,93.994,-36.4,4.857,5191.0
6,45,services,married,professional.course,no,yes,no,telephone,221,1,999,0,1.1,93.994,-36.4,4.857,5191.0
7,29,services,divorced,high.school,no,no,yes,telephone,251,11,999,0,1.1,93.994,-36.4,4.857,5191.0
8,39,unemployed,married,university.degree,no,no,no,telephone,132,2,999,0,1.4,94.465,-41.8,4.961,5228.1
9,35,admin.,married,high.school,unknown,yes,yes,telephone,236,1,999,0,1.1,93.994,-36.4,4.857,5191.0


In [146]:
#标准化，返回值为标准化后的数据
from sklearn.preprocessing import StandardScaler
StandardScaler().fit_transform(df_train.values)

array([[-0.96194567, -0.75441163, -0.28089438, ..., -1.31818749,
        -0.89471991, -0.35222574],
       [-0.08650311,  0.92377886,  1.35166813, ...,  0.71822332,
         0.34095638, -0.35222574],
       [-1.44830265,  0.92377886, -0.28089438, ...,  0.77974109,
         0.83979849, -0.35222574],
       ..., 
       [ 1.95619622,  0.36438203, -0.28089438, ..., -1.49814135,
        -2.71663653, -0.35222574],
       [-0.76740288,  1.48317569,  1.35166813, ...,  0.27035092,
         0.4054966 , -0.35222574],
       [ 2.63709599,  0.36438203, -0.28089438, ..., -1.55678446,
        -2.34015192, -0.35222574]])

In [183]:
train_X = df_train.values[:,:-1]
train_y = df_train.values[:,-1]
test_X  = df_test.values

In [190]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import RFE

#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
x = RFE(estimator=AdaBoostClassifier(), n_features_to_select=10).fit_transform(train_X, train_y)

In [194]:
clf = AdaBoostClassifier(n_estimators=1000)
clf.fit(x, train_y)
clf.predict(test_X)

ValueError: could not convert string to float: cellular